### Carga del modelo especializado desde HuggingFace

El siguiente bloque de código realiza la correspondiente carga del modelo previamente especializado y almacenado en HuggingFace.
También se carga el tokenizador correspondiente al modelo.

In [ ]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
new_model = "DrAleML/Oswestry-Instruct"
base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.bfloat16,
        return_dict=True,
      low_cpu_mem_usage=True,
        device_map="auto",
        trust_remote_code=True,
)
model = PeftModel.from_pretrained(base_model_reload, new_model)
tokenizer = AutoTokenizer.from_pretrained("DrAleML/Oswestry-Instruct")
modelo_path = "DrAleML/Oswestry-Instruct" 

### Script de Precisión basada en el conjunto invidual de ítems

Primero, se configura un pipeline de generación de texto que, dado un prompt de entrada (la entrevista), genera una respuesta del modelo. La respuesta generada es analizada mediante una función de extracción que utiliza expresiones regulares para identificar las puntuaciones asignadas a cada uno de los 10 ítems esperados de la escala (como "Intensidad dolor", "Andar", "Dormir", etc.).

A continuación, se evalúa el desempeño del modelo sobre un conjunto de datos de prueba (`test_set`). Para cada muestra, se construye un prompt con formato específico, se obtiene la salida generada por el modelo, y se comparan las puntuaciones extraídas con las puntuaciones reales esperadas. Las diferencias absolutas entre valores reales y predichos se registran para cada ítem.

Finalmente, se calcula y se imprime la media y la desviación estándar del error para cada uno de los ítems.


In [ ]:
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from tqdm import tqdm
import numpy as np


pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

NOMBRES_ESPERADOS = [
    "Intensidad dolor", "Estar pie", "Andar", "Cuidados personales", "Levantar peso",
    "Estar sentado", "Viajar", "Vida social", "Actividad sexual", "Dormir"
]

def extraer_items(texto):
    resultados = {}
    for nombre in NOMBRES_ESPERADOS:
        patron = re.search(rf"{nombre}.*?:\s*(\d+)\s*puntos?", texto, re.IGNORECASE)
        resultados[nombre] = int(patron.group(1)) if patron else -1
    return resultados

# Dataset de prueba 
test_set = [ # Aquí se establece el conjunto de datos de prueba
]


errores_por_item = {nombre: [] for nombre in NOMBRES_ESPERADOS}

for idx, muestra in enumerate(tqdm(test_set)):
    
    prompt = f"""<s>[INST] Clasifica la siguiente entrevista médico-paciente según la Escala Oswestry de discapacidad lumbar.\n\n{muestra['input']} [/INST]"""
    salida = pipe(prompt)[0]['generated_text']
    pred = extraer_items(salida)
    esperado = muestra['esperado']

    print(f"\n Entrevista {idx+1}")
    print("Respuesta generada:")
    print(salida)
    print("Diferencias por ítem:")

    for item in NOMBRES_ESPERADOS:
        real = esperado.get(item, -1)
        modelo = pred.get(item, -1)
        if real != -1 and modelo != -1:
            error = abs(real - modelo)
            errores_por_item[item].append(error)
            print(f" - {item:<20} | Esperado: {real} | Modelo: {modelo} | Diferencia: {error}")

# Resultados por ítem
print("\nEvaluación por ítem (promedio y desviación estándar de error)")
for item in NOMBRES_ESPERADOS:
    errores = errores_por_item[item]
    if errores:
        media = np.mean(errores)
        std = np.std(errores)
        print(f" - {item:<20} | Media error: {media:.2f} | Desviación estándar: {std:.2f}")
    else:
        print(f" - {item:<20} | Sin datos suficientes")

### Script de Precisión basada en la puntuación total por entrevista

Este script se centra en el cálculo del error total por entrevista.

Se utiliza un pipeline de generación de texto (`transformers`) previamente configurado, que toma como entrada una transcripción de entrevista y genera una clasificación automática en forma de texto. A partir de esa salida generada, se extraen las puntuaciones correspondientes a los 10 ítems de la escala Oswestry usando expresiones regulares.

Para cada muestra del conjunto de prueba (`test_set`), se construye un prompt en lenguaje natural que se pasa al modelo. Luego se comparan las puntuaciones generadas con las puntuaciones reales esperadas. En este caso, en lugar de evaluar ítem por ítem, se calcula la **suma total** de las puntuaciones reales y generadas por el modelo, y se registra el error absoluto entre ambas.

Al final, se imprime la **media** y la **desviación estándar** del error total por entrevista.

In [ ]:
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from tqdm import tqdm
import numpy as np

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

NOMBRES_ESPERADOS = [
    "Intensidad dolor", "Estar pie", "Andar", "Cuidados personales", "Levantar peso",
    "Estar sentado", "Viajar", "Vida social", "Actividad sexual", "Dormir"
]

def extraer_items(texto):
    resultados = {}
    for nombre in NOMBRES_ESPERADOS:
        patron = re.search(rf"{nombre}.*?:\s*(\d+)\s*puntos?", texto, re.IGNORECASE)
        resultados[nombre] = int(patron.group(1)) if patron else -1
    return resultados

# Dataset de prueba 
test_set = [ # Aquí se establece el conjunto de datos de prueba
]

errores = []

#Evaluación por entrevista completa

errores_por_entrevista = []

for idx, muestra in enumerate(tqdm(test_set)):
    prompt = f"""<s>[INST] Clasifica la siguiente entrevista médico-paciente según la Escala Oswestry de discapacidad lumbar.\n\n{muestra['input']} [/INST]"""
    salida = pipe(prompt)[0]['generated_text']
    pred = extraer_items(salida)
    esperado = muestra['esperado']

    total_real = sum(esperado.get(k, 0) for k in NOMBRES_ESPERADOS)
    total_modelo = sum(pred.get(k, 0) if pred.get(k, 0) != -1 else 0 for k in NOMBRES_ESPERADOS)
    error_total = abs(total_real - total_modelo)
    errores_por_entrevista.append(error_total)

    print(f"\nEntrevista {idx+1}")
    print("Respuesta generada:")
    print(salida)
    print(f"Puntuación esperada: {total_real} | Modelo: {total_modelo} | Diferencia: {error_total}")

print("\nEvaluación por entrevista completa")
print(f"Media de error total por entrevista: {np.mean(errores_por_entrevista):.2f} puntos")
print(f"Desviación estándar del error total: {np.std(errores_por_entrevista):.2f} puntos")